In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [14]:
import pandas as pd

file_path = 'climate_change_indicators.csv'
df = pd.read_csv(file_path)

years = [col for col in df.columns if col.startswith('F')]
df["total"] = df[years].sum('columns')

int_years = [int(year[1:]) for year in years]
global_avg_temp_change = df[years].mean()
global_avg_temp_change


F1961    0.163053
F1962   -0.013476
F1963   -0.006043
F1964   -0.070059
F1965   -0.247027
           ...   
F2018    1.302113
F2019    1.443061
F2020    1.552038
F2021    1.343531
F2022    1.382113
Length: 62, dtype: float64

In [15]:
!pip install ruptures --target=/kaggle/working/

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.4 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 291.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/474.0 kB ? eta -:--:--
   -- ------------------------------------- 30.7/474.0 kB ? eta -:--:--
   -- ------------------------------------- 30.7/474.0 kB ? eta -:--:--
   ----- --------------------------------- 61.4/474.0 kB 544.7 kB/s eta 0:00:01
   ----- --------------------------------- 61.4/474.0 kB 544.7 kB/s eta 0:00:01
   ----- ----------------------

In [2]:
import ruptures as rpt
import matplotlib.pyplot as plt

global_avg_temp_change_df = global_avg_temp_change.to_frame()

def plot_change_point_detection(df):
    algo = rpt.Pelt(model="l2")
    algo.fit(df)
    result = algo.predict(pen=1)
    rpt.display(df, [], result)
    plt.xticks(np.arange(len(int_years), step=3), int_years[::3], rotation=90)
    plt.xlabel("Year")
    plt.figtext(.05, .85, "Tipping points are indicated by dotted lines")
    plt.ylabel("Mean Temperature change");
plot_change_point_detection(global_avg_temp_change.to_frame())

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
from sklearn.cluster import KMeans
import matplotlib

model = KMeans(n_clusters=3)
model.fit(df.fillna(0)[years + ["total"]])

# Predicitng a single input
predicted_labels = model.predict(df.fillna(0)[years + ["total"]])
df["cluster"] = predicted_labels
climate_dynamic_clusters = df.groupby("cluster")[years].mean()
df.groupby("cluster")[years].mean().T.plot()
plt.xticks(np.arange(len(int_years), step=3), int_years[::3], rotation=90)

for idx in climate_dynamic_clusters.index:
    print("Cluster No. " + str(idx) + " has a total temperature change of: " + str(climate_dynamic_clusters.loc[idx].sum()))
    plot_change_point_detection(climate_dynamic_clusters.loc[idx].to_frame())